In [ ]:
import sys, os

In [ ]:
import getpass

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
print(module_path)

user = getpass.getuser()

if user == "sidsel":
    parquet_path = "/home/sidsel/workspace/sparkdata/parquet"
elif user == "svanhmic":
    parquet_path = "/home/svanhmic/workspace/data/DABAI/sparkdata/parquet"

In [ ]:
#import data!
from sample.DataIO import DataIO 
from shared.create_dummy_data import DummyData 
#data = DataIO(sc,feature_path=parquet_path+"/featureDataCvr.parquet",company_path=parquet_path+"/companyCvrData")
#feature_data = data.mergeCompanyFeatureData()
#small_data = feature_data.limit(50).cache()

dummy_data = DummyData(number_of_samples=20)
dummy_data.df.show()


In [ ]:
#Select parameters
from sample.CreateParameters import CreateParameters


params = CreateParameters(dummy_data.df.columns)

In [ ]:
params.select_params()

In [ ]:
#print(params.export_values())
dicKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}
dicBKM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'standardize': True, 'clusters': 2, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'BisectingKMeans', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': False, 'clusters': 2, 'tolerance': 0.0001}
dicGMM = {'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('x','y','z'), 'standardize': True, 'clusters': 10, 'tolerance': 0.0001}#{'initialstep': 10, 'optimizer': None, 'seed': 42, 'mindivisbleClusterSize': 1.0, 'model': 'GaussianMixture', 'initialmode': 'random', 'prediction': 'prediction', 'iterations': 10, 'probability': None, 'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'standardize': True, 'clusters': 10, 'tolerance': 0.0001}
#{'features': ('AarsVaerk_1', 'AarsVaerk_2', 'AarsVaerk_3', 'AarsVaerk_4'), 'initialstep': 10, 'standardize': True, 'clusters': 3, 'model': 'KMeans', 'initialmode': 'random', 'prediction': 'Prediction', 'iterations': 10}

In [ ]:
from sample.ExecuteWorkflow import ExecuteWorkflow
from random import random

import numpy as np

execution_model = ExecuteWorkflow()
#print(execution_model.params)
execution_model.params = dicBKM#params.export_values()

In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.linalg import Vectors,VectorUDT

udf_add_vector = F.udf(lambda x: Vectors.dense([random(),random(),random()]),VectorUDT())

runTest = execution_model.run(data=dummy_data.df).withColumn("dummy_vector",F.lit(udf_add_vector("label")))
runTest.show()

In [ ]:
bikmeans_with_negatives = BisectingKMeans(featuresCol="dummy_vector"
                           ,predictionCol="predictions"
                           #,maxIter=dicBKM["iterations"]
                           #,seed=dicBKM["seed"]
                           ,minDivisibleClusterSize=20
                           ,k=4
                          )

In [ ]:
#bikmeans_with_positives.fit(dataset=runTest)
model = bikmeans_with_negatives.fit(dataset=runTest)
model.transform(runTest).show()
model.computeCost(runTest)

In [ ]:
udfTypes = F.udf(lambda x: type(x))
runTest["label","scaled_features","casted_features"].show(truncate=False)

In [ ]:
from sample.ShowKmeans import ShowResults

results = ShowResults(execution_model.params)
results.select_prototypes(runTest)